In [ ]:
from IPython.display import display
import fileupload
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
from ipywidgets import widgets

uploader = fileupload.FileUploadWidget()
rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
textBox = widgets.Text()
button = widgets.Button(description="Buscar")
label = widgets.Label(value='Introduzca la palabra que desea buscar')

def _handle_upload(change):
    w = change['owner']
    with open(w.filename, 'wb') as f:
        f.write(w.data)
    pdfLocal = w.filename
    fp = file(pdfLocal, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    global paragraphs
    paragraphs = text.split(".\n")
    display(label)
    display(textBox)
    display(button)
    
    
def handle_submit(sender):
    palabra_buscada = textBox.value
    palabras = [None] * len(paragraphs)
    for paragraph in paragraphs:
        palabras[paragraphs.index(paragraph)] = paragraph.split(" ");
    contador = 0
    for parrafo in palabras:
        contadorParrafo = 0
        for palabra in parrafo:
            if(palabra_buscada.encode('utf8') in palabra):
                contador += 1
                contadorParrafo+=1
        print 'la palabra "%s" aparece %d veces en el parrafo %d' % (palabra_buscada,contadorParrafo, palabras.index(parrafo))
    print '\nla palabra "%s" aparece %d veces en total' % (palabra_buscada,contador)



uploader.observe(_handle_upload, names='data')
button.on_click(handle_submit)

display(uploader)